# Experiment 1: Oracle vs Reference

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import Tuple, List
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from torch.nn.functional import cross_entropy
from torch.utils.data import random_split, DataLoader
from tqdm import tqdm

from src.reference import RotatedMNISTClassifier
from src.data import RotatedMNISTDataset, FixedSizeWrapper

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Helper Functions

In [8]:
def do_train_epoch(model, loader, optimizer, epoch) -> float:
    model.train()
    epoch_losses = []
    for idx, (images, rotation_labels, digit_labels) in tqdm(enumerate(loader)):
        images = images.to(device)
        rotation_labels = rotation_labels.to(device)
        digit_labels = digit_labels.to(device)

        optimizer.zero_grad()
        logits = reference_model(images) # (bs, num_digit_classes)
        loss = cross_entropy(logits, digit_labels)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
    epoch_loss = torch.mean(torch.Tensor(epoch_losses)).item()
    print(f'Train epoch {epoch} loss: {epoch_loss:.4f}')
    return epoch_loss

def do_val_epoch(model, loader, epoch) -> float:
    model.eval()
    epoch_losses = []
    for idx, (images, rotation_labels, digit_labels) in tqdm(enumerate(loader)):
        images = images.to(device)
        rotation_labels = rotation_labels.to(device)
        digit_labels = digit_labels.to(device)

        logits = reference_model(images) # (bs, num_digit_classes)
        loss = cross_entropy(logits, digit_labels)
        epoch_losses.append(loss.item())
    epoch_loss = torch.mean(torch.Tensor(epoch_losses)).item()
    print(f'Val epoch {epoch} loss: {epoch_loss:.4f}') 
    return epoch_loss

def get_accuracy(model, loader):
    total_samples = 0
    total_correct = 0
    for idx, (images, rotation_labels, digit_labels) in tqdm(enumerate(loader)):
        images = images.to(device)
        rotation_labels = rotation_labels.to(device)
        digit_labels = digit_labels.to(device) # (bs,)

        logits = reference_model(images) # (bs, num_digit_classes)
        prediction = torch.argmax(logits, dim=1) # (bs,)
        total_samples += prediction.shape[0]
        total_correct += torch.sum(prediction == digit_labels)
    accuracy = total_correct / total_samples * 100
    return accuracy

def get_rotated_mnist_loaders(downsample_factor: int = 20) -> Tuple[DataLoader, DataLoader, DataLoader]:
    # Initialize dataset
    dataset = RotatedMNISTDataset()
    
    # Assuming `dataset` is your PyTorch Dataset
    dataset_size = len(dataset)
    train_size = int(0.7 * dataset_size)
    val_size = int(0.2 * dataset_size)
    test_size = dataset_size - train_size - val_size
    
    train_dataset, val_dataset, test_dataset = random_split(
        dataset, [train_size, val_size, test_size], 
        generator=torch.Generator().manual_seed(40)
    )
    
    train_dataset = FixedSizeWrapper(dataset = train_dataset, size = train_size // downsample_factor)
    val_dataset = FixedSizeWrapper(dataset = val_dataset, size = val_size // downsample_factor)
    test_dataset = FixedSizeWrapper(dataset = test_dataset, size = test_size // downsample_factor)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    return train_loader, val_loader, test_loader

def training_loop(model: nn.Module, train_loader: DataLoader, val_loader: DataLoader, test_loader: DataLoader, num_epochs: int = 100, lr: float = 0.005, epochs_per_accuracy: int = 5) -> List[float]:
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    val_losses = []
    for epoch in range(num_epochs):
        do_train_epoch(model, train_loader, optimizer, epoch)
        val_loss = do_val_epoch(model, val_loader, epoch)
        val_losses.append(val_loss)
        if epoch % epochs_per_accuracy == 0:
            accuracy = get_accuracy(model, test_loader)
            print(f'Test accuracy: {accuracy:.3f}%')
    return val_losses

In [ ]:
train_loader, val_loader, test_loader = get_rotated_mnist_loaders()

# Training Loop for Reference Model

In [9]:
# Training loop
reference_model = RotatedMNISTClassifier().to(device)
val_losses = training_loop(
    model=reference_model, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    test_loader=test_loader, 
    num_epochs=50, 
    lr = 0.005, 
    epochs_per_accuracy=5)

263it [00:02, 122.53it/s]


Train epoch 0 loss: 1.1081


75it [00:00, 177.44it/s]


Val epoch 0 loss: 0.6816


38it [00:00, 177.33it/s]


...

Val epoch 48 loss: 0.2231


263it [00:02, 126.62it/s]


Train epoch 49 loss: 0.1475


75it [00:00, 178.66it/s]

Val epoch 49 loss: 0.1893


In [13]:
get_accuracy(reference_model, test_loader)

38it [00:00, 144.18it/s]


tensor(94.4167, device='cuda:0')

# Training Loop for GoE with Oracle Router

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from src.binary_tree import BinaryTreeGoE, MNISTOracleRouter

In [15]:
# Initialize 
class LayerOne(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1, 32, kernel_size=3, padding=1)

    def forward(self, x: torch.Tensor):
        """
        Args:
        - x: (batch_size, 32, 28, 28)
        """
        return F.relu(F.max_pool2d(self.conv(x), 2))  # (batch_size, 32, 14, 14)

class LayerTwo(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(32, 64, kernel_size=3, padding=1)

    def forward(self, x: torch.Tensor):
        """
        Args:
        - x: (batch_size, 32, 14, 14)
        """
        return F.relu(F.max_pool2d(self.conv(x), 2))  # (batch_size, 64, 7, 7)

class LayerThree(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x: torch.Tensor):
        """
        Args:
        - x: (batch_size, 32, 7, 7)
        """
        x = x.view(-1, 64 * 7 * 7)  # (batch_size, 64 * 7 * 7)
        x = F.relu(self.fc1(x))  # (batch_size, 128)
        logits = self.fc2(x)  # (batch_size, 10)
        return logits

In [16]:
# Training loop
router = MNISTOracleRouter()
goe_model = BinaryTreeGoE(modules_by_depth = [LayerOne, LayerTwo, LayerThree], router=router)
val_losses = training_loop(
    model=goe_model, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    test_loader=test_loader, 
    num_epochs=50, 
    lr = 0.005, 
    epochs_per_accuracy=5)

263it [00:02, 115.48it/s]


Train epoch 0 loss: 0.1537


75it [00:00, 176.35it/s]


Val epoch 0 loss: 0.1424


38it [00:00, 179.00it/s]


...

Val epoch 48 loss: 0.1788


263it [00:02, 129.55it/s]


Train epoch 49 loss: 0.1603


75it [00:00, 183.13it/s]

Val epoch 49 loss: 0.1580


In [17]:
get_accuracy(goe_model, test_loader)

38it [00:00, 107.82it/s]


tensor(95.1667, device='cuda:0')

In [22]:
from src.metrics import get_model_flops

get_model_flops(goe_model, next(iter(train_loader))[0])

TypeError: get_path() missing 1 required positional argument: 'rotation_class'

In [ ]:
from matplotlib import pyplot as plt

# Sample images from each set
num_samples_per_set = 10
_, axes = plt.subplots(3, 10, figsize=(10 * 10, 10 * 3))
axes = axes.reshape((3,10))

for curr_idx, curr_dataset in enumerate((train_dataset, val_dataset, test_dataset)):
    for sample_idx in range(num_samples_per_set):
        img, rotation_label, digit_label = curr_dataset[sample_idx]
        ax = axes[curr_idx, sample_idx]
        img = img.squeeze().numpy()
        ax.imshow(img, cmap='gray')
        ax.set_title(f'Rotation: {rotation_label * 90}°, Digit: {digit_label}')
        ax.axis('off')

# Adjust layout and display
# plt.tight_layout()
plt.show()